In [5]:
from google.colab import files

# This will open a "Choose Files" dialog in your browser
print("Please select your water_dataset.zip file for upload...")
uploaded = files.upload()

# Print a confirmation message once it's done
for fn in uploaded.keys():
  print(f'Successfully uploaded file "{fn}"')

Please select your water_dataset.zip file for upload...


Saving water_images.zip to water_images.zip
Successfully uploaded file "water_images.zip"


In [18]:
# Make sure the filename matches what you uploaded
!unzip -q water_images.zip

replace water_images/test/algae_bloom/1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [19]:
# 'ls' is a command to "list" files and directories
# -R means "recursive" (show me all folders inside)
!ls -R water_images

water_images:
test  train

water_images/test:
algae_bloom  healthy_water

water_images/test/algae_bloom:
10.jpg	12.jpg	14.jpg	1.jpg  3.jpg  5.jpg  7.jpg  9.jpg
11.jpg	13.jpg	15.jpg	2.jpg  4.jpg  6.jpg  8.jpg

water_images/test/healthy_water:
10.jpg	12.jpg	14.jpg	1.jpg  3.jpg  5.jpg  7.jpg  9.jpg
11.jpg	13.jpg	15.jpg	2.jpg  4.jpg  6.jpg  8.jpg

water_images/train:
algae_bloom  healthy_water

water_images/train/algae_bloom:
51.jpg	63.jpg	75.jpg	     image15.jpg  image26.jpg  image37.jpg  image48.jpg
52.jpg	64.jpg	76.jpg	     image16.jpg  image27.jpg  image38.jpg  image49.jpg
53.jpg	65.jpg	77.jpg	     image17.jpg  image28.jpg  image39.jpg  image4.jpg
54.jpg	66.jpg	78.jpg	     image18.jpg  image29.jpg  image3.jpg   image50.jpg
55.jpg	67.jpg	79.jpg	     image19.jpg  image2.jpg   image40.jpg  image5.jpg
56.jpg	68.jpg	80.jpg	     image1.jpg   image30.jpg  image41.jpg  image6.jpg
57.jpg	69.jpg	iamge14.jpg  image20.jpg  image31.jpg  image42.jpg  image7.jpg
58.jpg	70.jpg	iimage6.jpg  image21.jpg

In [20]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# --- Our Project's "Settings" ---

# Define the size to resize all images to
IMG_SIZE = (150, 150)
BATCH_SIZE = 16  # How many images to process at a time

# Define the paths based on your unzipped folder name
TRAIN_DIR = 'water_images/train'
TEST_DIR = 'water_images/test'

In [21]:
# 1. Create augmentation generator
train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# 2. Create test generator
test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# 3. Create the actual "generators"
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

validation_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 157 images belonging to 2 classes.
Found 30 images belonging to 2 classes.


In [22]:
model = keras.Sequential([
    layers.Input(shape=IMG_SIZE + (3,)),

    # Block 1
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    # Block 2
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    # Block 3
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    # Classifier Head
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

print(model.summary())

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)               │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │     4,735,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,828,481 (18.42 MB)

 Trainable params: 4,828,481 (18.42 MB)

 Non-trainable params: 0 (0.00 B)

None


In [23]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print("Model compiled successfully.")


Model compiled successfully.


In [24]:
print("Starting model training...")

history = model.fit(
    train_generator,
    epochs=15,
    validation_data=validation_generator
)

print("Model training complete.")

Starting model training...


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 12s 997ms/step - accuracy: 0.5887 - loss: 1.2576 - val_accuracy: 0.9333 - val_loss: 0.4782
Epoch 2/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 8s 796ms/step - accuracy: 0.9029 - loss: 0.3474 - val_accuracy: 0.8000 - val_loss: 1.0769
Epoch 3/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 9s 885ms/step - accuracy: 0.9393 - loss: 0.1594 - val_accuracy: 0.8333 - val_loss: 0.5264
Epoch 4/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 9s 932ms/step - accuracy: 0.9301 - loss: 0.1529 - val_accuracy: 0.8333 - val_loss: 0.3779
Epoch 5/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 9s 866ms/step - accuracy: 0.9555 - loss: 0.0775 - val_accuracy: 0.9667 - val_loss: 0.0737
Epoch 6/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 10s 793ms/step - accuracy: 0.9920 - loss: 0.0363 - val_accuracy: 0.9667 - val_loss: 0.1080
Epoch 7/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 9s 936ms/step - accuracy: 0.9790 - loss: 0.0809 - val_accuracy: 0.9667 - val_loss: 0.1079
Epoch 8/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 9s 924ms/step - accuracy: 0.9751 - loss: 0.0870 - val_accuracy: 